# Practical Application III: Comparing Classifiers

**Overview**: In this practical application, your goal is to compare the performance of the classifiers we encountered in this section, namely K Nearest Neighbor, Logistic Regression, Decision Trees, and Support Vector Machines.  We will utilize a dataset related to marketing bank products over the telephone.  



### Getting Started

Our dataset comes from the UCI Machine Learning repository [link](https://archive.ics.uci.edu/ml/datasets/bank+marketing).  The data is from a Portugese banking institution and is a collection of the results of multiple marketing campaigns.  We will make use of the article accompanying the dataset [here](CRISP-DM-BANK.pdf) for more information on the data and features.



### Problem 1: Understanding the Data

To gain a better understanding of the data, please read the information provided in the UCI link above, and examine the **Materials and Methods** section of the paper.  How many marketing campaigns does this data represent?

The data represents the latest marketing campain which involved getting potential clients to subscribe to a long term deposit application with what the bank considered a good interest rate.  Furthermore, there was some information retained from the previous campain for each person contacted, which included the previous compains success.

### Problem 2: Read in the Data

Use pandas to read in the dataset `bank-additional-full.csv` and assign to a meaningful variable name.

#### Dataset Characteristics
- Data is imbalanced with regards to using the success vs failure class/column 'y': 
  - No  : 36548
  - Yes : 4640
- No NaN or empty feature/column values detected
- 

In [63]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder, PolynomialFeatures
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier, Lasso
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
import time

In [69]:
df = pd.read_csv('data/bank-additional-full.csv', sep = ';')

df.columns = ['age', 'job_type', 'marital_status', 'education', 'credit_in_default', 'housing_loan', 'personal_loan', 'contact_method', 'month_last_contact', 'dayOfWeek_last_contact', 'contact_duration_seconds', 'contact_attempts_during_campaign', 'days_since_last_contact', 'contact_attempts_previous_camapain', 'outcome_previous_campain', 'employment_variation_rate_quarteryly', 'consumer_price_index_monthly', 'consumer_confidence_index_monthly', 'euribor3_monthly_rate_daily', 'number_employed_quarterly', 'subscribed']

In [70]:
# dataset balance of success class ('column: y').
df['subscribed'].value_counts()

subscribed
no     36548
yes     4640
Name: count, dtype: int64

In [ ]:
# X = df.drop('subscribed', axis=1)
# y = df.subscribed
# categorical_columns = X.select_dtypes(include=['object']).columns.tolist()
# numerical_columns = X.select_dtypes(include=['float64', 'int64']).columns.tolist()

# print('categorical columns: ', categorical_columns)
# print('numeric columns: ', numerical_columns)

In [71]:
df.head(-10)

,age,job_type,marital_status,education,credit_in_default,housing_loan,personal_loan,contact_method,month_last_contact,dayOfWeek_last_contact,...,contact_attempts_during_campaign,days_since_last_contact,contact_attempts_previous_camapain,outcome_previous_campain,employment_variation_rate_quarteryly,consumer_price_index_monthly,consumer_confidence_index_monthly,euribor3_monthly_rate_daily,number_employed_quarterly,subscribed
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41173,62,retired,married,university.degree,no,yes,no,cellular,nov,thu,...,1,999,2,failure,-1.1,94.767,-50.8,1.031,4963.6,yes
41174,62,retired,married,university.degree,no,yes,no,cellular,nov,thu,...,1,1,6,success,-1.1,94.767,-50.8,1.031,4963.6,yes
41175,34,student,single,unknown,no,yes,no,cellular,nov,thu,...,1,999,2,failure,-1.1,94.767,-50.8,1.031,4963.6,no
41176,38,housemaid,divorced,high.school,no,yes,yes,cellular,nov,thu,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.031,4963.6,no


In [72]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   age                                   41188 non-null  int64  
 1   job_type                              41188 non-null  object 
 2   marital_status                        41188 non-null  object 
 3   education                             41188 non-null  object 
 4   credit_in_default                     41188 non-null  object 
 5   housing_loan                          41188 non-null  object 
 6   personal_loan                         41188 non-null  object 
 7   contact_method                        41188 non-null  object 
 8   month_last_contact                    41188 non-null  object 
 9   dayOfWeek_last_contact                41188 non-null  object 
 10  contact_duration_seconds              41188 non-null  int64  
 11  contact_attempt

### Problem 3: Understanding the Features


Examine the data description below, and determine if any of the features are missing values or need to be coerced to a different data type.


```
Input variables:
# bank client data:
1 - age (numeric)
2 - job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')
3 - marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)
4 - education (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')
5 - default: has credit in default? (categorical: 'no','yes','unknown')
6 - housing: has housing loan? (categorical: 'no','yes','unknown')
7 - loan: has personal loan? (categorical: 'no','yes','unknown')
# related with the last contact of the current campaign:
8 - contact: contact communication type (categorical: 'cellular','telephone')
9 - (Month of Contact) - month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
10 - day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')
11 - (Call Duration) - duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.
# other attributes:
12 - (Number of Previous Contacts) - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
13 - (Days Since Last Contact) - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
14 - previous: number of contacts performed before this campaign and for this client (numeric)
15 - (Last Contact Results) - poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')
# social and economic context attributes
16 - emp.var.rate: employment variation rate - quarterly indicator (numeric)
17 - cons.price.idx: consumer price index - monthly indicator (numeric)
18 - cons.conf.idx: consumer confidence index - monthly indicator (numeric)
19 - euribor3m: euribor 3 month rate - daily indicator (numeric)
20 - nr.employed: number of employees - quarterly indicator (numeric)

Output variable (desired target):
21 - y - has the client subscribed a term deposit? (binary: 'yes','no')
```

The following columns would be transformed using the OneHotEncoder.  Furthermore, any columns generated from data that equaled the value of 'unknown' would be removed.

In [76]:
# get column names for numeric and catagorical features
categorical_columns = df.drop('subscribed', axis=1).select_dtypes(include=['object']).columns.tolist()
numerical_columns = df.select_dtypes(include=['float64', 'int64']).columns.tolist()

# features that require OneHotEncoding
for category_name in categorical_columns :
    catagory_count = df[category_name].value_counts()
    print (catagory_count)
    print ('\n')



job_type
admin.           10422
blue-collar       9254
technician        6743
services          3969
management        2924
retired           1720
entrepreneur      1456
self-employed     1421
housemaid         1060
unemployed        1014
student            875
unknown            330
Name: count, dtype: int64


marital_status
married     24928
single      11568
divorced     4612
unknown        80
Name: count, dtype: int64


education
university.degree      12168
high.school             9515
basic.9y                6045
professional.course     5243
basic.4y                4176
basic.6y                2292
unknown                 1731
illiterate                18
Name: count, dtype: int64


credit_in_default
no         32588
unknown     8597
yes            3
Name: count, dtype: int64


housing_loan
yes        21576
no         18622
unknown      990
Name: count, dtype: int64


personal_loan
no         33950
yes         6248
unknown      990
Name: count, dtype: int64


contact_method
cellu

In [ ]:
print('categorical columns: ', categorical_columns)

# create feature DataFrame
X = df.drop('subscribed', axis=1)

# encode columns with category data and remove unwanted columns
for category_name in categorical_columns :
    encoder = OneHotEncoder() # drop='first'...doing this manually

    # transform category data
    encoded_data = encoder.fit_transform(X[[category_name]])

    # Convert the encoded data to a DataFrame
    encoded_df = pd.DataFrame(encoded_data.toarray(), columns=encoder.get_feature_names_out([category_name]))

    # Drop OHE created: 'unknown' categorical column from DataFrame...this replaces drop='first' for OHE
    if category_name + '_unknown' in encoded_df.columns:
        encoded_df.drop([category_name + '_unknown'], axis=1, inplace=True)

    elif encoded_df.columns.size > 2: 
        # drop first column if 'unknown' column doesn't exist and there are more than two columns total generated by OHE
        encoded_df.drop([encoded_df.columns[0]], axis=1, inplace=True) 

    print('remaining columns from encoder for category ['+ category_name +']: ', encoded_df.columns.tolist())

    # Concatenate the encoded DataFrame with the original DataFrame
    X = pd.concat([X, encoded_df], axis=1)

    # Drop original categorical column from DataFrame
    X.drop([category_name], axis=1, inplace=True)



categorical columns:  ['job_type', 'marital_status', 'education', 'credit_in_default', 'housing_loan', 'personal_loan', 'contact_method', 'month_last_contact', 'dayOfWeek_last_contact', 'outcome_previous_campain']
remaining columns from encoder for category [job_type]:  ['job_type_admin.', 'job_type_blue-collar', 'job_type_entrepreneur', 'job_type_housemaid', 'job_type_management', 'job_type_retired', 'job_type_self-employed', 'job_type_services', 'job_type_student', 'job_type_technician', 'job_type_unemployed']
remaining columns from encoder for category [marital_status]:  ['marital_status_divorced', 'marital_status_married', 'marital_status_single']
remaining columns from encoder for category [education]:  ['education_basic.4y', 'education_basic.6y', 'education_basic.9y', 'education_high.school', 'education_illiterate', 'education_professional.course', 'education_university.degree']
remaining columns from encoder for category [credit_in_default]:  ['credit_in_default_no', 'credit_in_

### Problem 4: Understanding the Task

After examining the description and data, your goal now is to clearly state the *Business Objective* of the task.  State the objective below.

### Problem 5: Engineering Features

Now that you understand your business objective, we will build a basic model to get started.  Before we can do this, we must work to encode the data.  Using just the bank information features, prepare the features and target column for modeling with appropriate encoding and transformations.

In [83]:
# OneHotEncoding performed in previous step...here only need standard scaler
preprocessor = ColumnTransformer(
    transformers=[
        ('encoder', StandardScaler(), X.columns.tolist())
    ])

# extract target
y = df.subscribed

# Encode the target variable
le = LabelEncoder()
y = le.fit_transform(y)


### Problem 6: Train/Test Split

With your data prepared, split it into a train and test set.

In [84]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Problem 7: A Baseline Model

Before we build our first model, we want to establish a baseline.  What is the baseline performance that our classifier should aim to beat?

In [ ]:
# the baseline models using default hyperparameters
models = {
    'knn': (KNeighborsClassifier()),
    'logisticregression': (LogisticRegression()),
    'svc': (SVC()),
    'decisiontreeclassifier': (DecisionTreeClassifier())
}


In [86]:
results = []

for name, (model) in models.items():
    # Create a pipeline
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        (name, model)
    ])
    
    # Perform grid search
    grid_search = GridSearchCV(pipeline, param_grid={}, cv=5, n_jobs=-1)
    
    # Fit the model and time it
    start_time = time.time()
    grid_search.fit(X_train, y_train)
    fit_time = (time.time() - start_time) / len(grid_search.cv_results_['mean_fit_time'])
    
    # Get the best estimator
    best_model = grid_search.best_estimator_
    print(grid_search.best_params_)
    print(grid_search.best_estimator_)
    
    # Evaluate on training and test sets
    train_score = best_model.score(X_train, y_train)
    test_score = best_model.score(X_test, y_test)
    
    # Append the results
    results.append([name, train_score, test_score, fit_time])

# Create the results DataFrame
results_df = pd.DataFrame(results, columns=['model', 'train score', 'test score', 'average fit time'])
results_df.set_index('model', inplace=True)

results_df.to_json('data/baseline_model_results.json')

{}
Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('encoder', StandardScaler(),
                                                  ['age',
                                                   'contact_duration_seconds',
                                                   'contact_attempts_during_campaign',
                                                   'days_since_last_contact',
                                                   'contact_attempts_previous_camapain',
                                                   'employment_variation_rate_quarteryly',
                                                   'consumer_price_index_monthly',
                                                   'consumer_confidence_index_monthly',
                                                   'euribor3_monthly_ra...
                                                   'job_type_self-employed',
                                                   'job_type_services',
        

### Problem 8: A Simple Model

Use Logistic Regression to build a basic model on your data.  

In [ ]:
#Logistic regregession was used to model data in previous step

### Problem 9: Score the Model

What is the accuracy of your model?

### Problem 10: Model Comparisons

Now, we aim to compare the performance of the Logistic Regression model to our KNN algorithm, Decision Tree, and SVM models.  Using the default settings for each of the models, fit and score each.  Also, be sure to compare the fit time of each of the models.  Present your findings in a `DataFrame` similar to that below:

| Model | Train Time | Train Accuracy | Test Accuracy |
| ----- | ---------- | -------------  | -----------   |
|     |    |.     |.     |

In [88]:
baseline_df = pd.read_json(r'data/baseline_model_results.json')
baseline_df

,train score,test score,average fit time
knn,0.920759,0.896091,7.879680
logisticregression,0.911806,0.911265,3.054467
svc,0.926555,0.908230,41.119052
decisiontreeclassifier,1.000000,0.883103,2.874049


### Problem 11: Improving the Model

Now that we have some basic models on the board, we want to try to improve these.  Below, we list a few things to explore in this pursuit.

- More feature engineering and exploration.  For example, should we keep the gender feature?  Why or why not?
- Hyperparameter tuning and grid search.  All of our models have additional hyperparameters to tune and explore.  For example the number of neighbors in KNN or the maximum depth of a Decision Tree.  
- Adjust your performance metric

##### Questions